In [2]:
import zipfile
import os

# Path to your ZIP file (update this)
zip_path = "/content/archive.zip"
extract_path = "C:\\Users\\rithv\\OneDrive\\Desktop"

# Extract the ZIP file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("✅ ZIP file extracted successfully!")


✅ ZIP file extracted successfully!


In [3]:
import os
import shutil
import random

# Set dataset path (CORRECTED)
# dataset_path = "/content/C:/Users/YourName/Documents/dataset/Brain_Tumor_Dataset"  # Incorrect path
dataset_path = "/content/C:\\Users\\rithv\\OneDrive\\Desktop/Brain_Tumor_Dataset"  # Corrected: Remove '/content/'
output_dir = "C:/Users/YourName/Documents/dataset_split"  # Where train & test will be stored

# Define train & test paths
train_path = os.path.join(output_dir, "train")
test_path = os.path.join(output_dir, "test")

# Create train & test directories
os.makedirs(train_path, exist_ok=True)
os.makedirs(test_path, exist_ok=True)

# Loop through categories (positive & negative)
for category in ["Positive", "Negative"]:
    class_dir = os.path.join(dataset_path, category)
    train_class_dir = os.path.join(train_path, category)
    test_class_dir = os.path.join(test_path, category)

    os.makedirs(train_class_dir, exist_ok=True)
    os.makedirs(test_class_dir, exist_ok=True)

    # Get all image files
    images = [img for img in os.listdir(class_dir) if img.lower().endswith(('.png', '.jpg', '.jpeg'))]
    random.shuffle(images)  # Shuffle images for randomness

    # Split 80% train, 20% test
    split_idx = int(0.8 * len(images))
    train_images = images[:split_idx]
    test_images = images[split_idx:]

    # Copy images into respective folders
    for img in train_images:
        shutil.copy(os.path.join(class_dir, img), os.path.join(train_class_dir, img))

    for img in test_images:
        shutil.copy(os.path.join(class_dir, img), os.path.join(test_class_dir, img))

print("✅ Dataset split completed! Check the 'dataset_split' folder.")

✅ Dataset split completed! Check the 'dataset_split' folder.


In [4]:
import shutil

# Path to the split dataset folder (update this)
split_dataset_path = "/content/C:/Users/YourName/Documents/dataset_split"

# Path to save the ZIP file
zip_output_path = "/content/C:\\Users\\rithv\\OneDrive\\Desktop"

# Create a ZIP file
shutil.make_archive(zip_output_path.replace(".zip", ""), 'zip', split_dataset_path)

print(f"✅ Dataset zipped successfully! Saved at: {zip_output_path}")


✅ Dataset zipped successfully! Saved at: /content/C:\Users\rithv\OneDrive\Desktop


In [5]:
import os

# Path to the split dataset (UPDATE THIS)
split_dataset_path = "/content/C:/Users/YourName/Documents/dataset_split"

# Loop through train and test folders
for folder in ["train", "test"]:
    print(f"\n📂 {folder.upper()} SET:")
    for category in ["Positive", "Negative"]:
        category_path = os.path.join(split_dataset_path, folder, category)
        num_files = len(os.listdir(category_path))
        print(f"  - {category}: {num_files} images")



📂 TRAIN SET:
  - Positive: 2612 images
  - Negative: 1600 images

📂 TEST SET:
  - Positive: 654 images
  - Negative: 400 images


PREPROCESSING

In [6]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [7]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                            shear_range=0.2,
                            zoom_range=0.2,
                            horizontal_flip=True)
train_dataset = train_datagen.flow_from_directory('/content/C:/Users/YourName/Documents/dataset_split/train',target_size=(64,64),batch_size=32,class_mode='binary')

Found 4212 images belonging to 2 classes.


In [8]:
import shutil
import os

train_path = "C:/Users/YourName/Documents/dataset_split/train/.ipynb_checkpoints"

# Check if the folder exists, then delete it
if os.path.exists(train_path):
    shutil.rmtree(train_path)
    print("✅ Removed .ipynb_checkpoints folder!")
else:
    print("No .ipynb_checkpoints folder found.")


No .ipynb_checkpoints folder found.


In [9]:
print(train_dataset.class_indices)  # This will show detected class names

{'Negative': 0, 'Positive': 1}


In [10]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_dataset = test_datagen.flow_from_directory('/content/C:/Users/YourName/Documents/dataset_split/test',target_size=(64,64),batch_size=32,class_mode='binary')

Found 1054 images belonging to 2 classes.


In [11]:
import shutil
import os

test_path = "C:/Users/YourName/Documents/dataset_split/test/.ipynb_checkpoints"

# Check if the folder exists, then delete it
if os.path.exists(test_path):
    shutil.rmtree(test_path)
    print("✅ Removed .ipynb_checkpoints folder!")
else:
    print("No .ipynb_checkpoints folder found.")


No .ipynb_checkpoints folder found.


In [12]:
print(test_dataset.class_indices)  # This will show detected class names


{'Negative': 0, 'Positive': 1}


BUILDING CNN

In [13]:
cnn = tf.keras.models.Sequential()

In [14]:
Input_Shape=(64,64,3)

In [15]:
cnn.add(tf.keras.layers.Input(shape=Input_Shape))
cnn.add(tf.keras.layers.Conv2D(filters=32,kernel_size=3,activation="relu"))

In [16]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2,strides=2))

In [17]:
cnn.add(tf.keras.layers.Conv2D(filters=32,kernel_size=3,activation="relu"))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2,strides=2))

In [18]:
cnn.add(tf.keras.layers.Flatten())

In [19]:
cnn.add(tf.keras.layers.Dense(units=150,activation="relu"))

In [20]:
cnn.add(tf.keras.layers.Dense(units=50,activation="relu"))

In [21]:
cnn.add(tf.keras.layers.Dense(units=1,activation="sigmoid"))

In [22]:
cnn.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])

In [23]:
cnn.fit(x=train_dataset,validation_data=test_dataset,epochs=30)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30
132/132 ━━━━━━━━━━━━━━━━━━━━ 34s 240ms/step - accuracy: 0.7745 - loss: 0.4688 - val_accuracy: 0.8928 - val_loss: 0.3180
Epoch 2/30
132/132 ━━━━━━━━━━━━━━━━━━━━ 34s 256ms/step - accuracy: 0.9115 - loss: 0.2225 - val_accuracy: 0.9070 - val_loss: 0.2492
Epoch 3/30
132/132 ━━━━━━━━━━━━━━━━━━━━ 33s 244ms/step - accuracy: 0.9438 - loss: 0.1534 - val_accuracy: 0.9250 - val_loss: 0.2159
Epoch 4/30
132/132 ━━━━━━━━━━━━━━━━━━━━ 36s 274ms/step - accuracy: 0.9487 - loss: 0.1490 - val_accuracy: 0.9440 - val_loss: 0.1410
Epoch 5/30
132/132 ━━━━━━━━━━━━━━━━━━━━ 33s 250ms/step - accuracy: 0.9510 - loss: 0.1292 - val_accuracy: 0.9023 - val_loss: 0.2827
Epoch 6/30
132/132 ━━━━━━━━━━━━━━━━━━━━ 32s 238ms/step - accuracy: 0.9671 - loss: 0.0959 - val_accuracy: 0.9516 - val_loss: 0.1311
Epoch 7/30
132/132 ━━━━━━━━━━━━━━━━━━━━ 30s 227ms/step - accuracy: 0.9557 - loss: 0.1090 - val_accuracy: 0.9459 - val_loss: 0.1447
Epoch 8/30
132/132 ━━━━━━━━━━━━━━━━━━━━ 31s 234ms/step - accuracy: 0.9654 - loss: 0

In [24]:
import numpy as np
from tensorflow.keras.preprocessing import image
test_image = image.load_img('/content/pos3.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image,axis=0)
result = cnn.predict(test_image)
prediction=""
if result[0][0]==0:
    prediction="Negative"
else:
    prediction="Positive"

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step


In [25]:
result

array([[1.]], dtype=float32)

In [26]:
import keras
keras.saving.save_model(cnn,"Brain_Tumour.keras")

In [27]:
from google.colab import files
files.download("Brain_Tumour.keras")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>